### 0. Imports and requirements

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score


from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook

pd.set_option('display.max_columns', None)

### 1. Data Preprocessing

In [2]:
TRAIN_TRANSACTIONS_PATH = '../input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_train_transactions_contest/train_transactions_contest'
TEST_TRANSACTIONS_PATH = '../input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_test_transactions_contest/test_transactions_contest'

TRAIN_TARGET_PATH = '../input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_train_target.csv'

In [3]:
target_frame = pd.read_csv(TRAIN_TARGET_PATH)
target_frame.head()

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


In [4]:
def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразует их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset) 
                              if filename.startswith('part')])
    
    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

In [5]:
from typing import Dict

features = ['currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group', 'ecommerce_flag',
            'payment_system', 'income_flag', 'mcc', 'country', 'city', 'mcc_category', 'day_of_week',
            'hour', 'weekofyear', 'amnt', 'days_before', 'hour_diff']

def pad_sequence(array, max_len) -> np.array:
    """
    принимает список списков (array) и делает padding каждого вложенного списка до max_len
    :param array: список списков
    :param max_len: максимальная длина до которой нужно сделать padding
    :return: np.array после padding каждого вложенного списка до одинаковой длины
    """
    add_zeros = max_len - len(array[0])
    return np.array([list(x) + [0] * add_zeros for x in array])


def truncate(x, num_last_transactions=750):
    return x.values.transpose()[:, -num_last_transactions:].tolist()


def transform_transactions_to_sequences(transactions_frame: pd.DataFrame,
                                        num_last_transactions=750) -> pd.DataFrame:
    """
    принимает frame с транзакциями клиентов, сортирует транзакции по клиентам
    (внутри клиента сортирует транзакции по возрастанию), берет num_last_transactions танзакций,
    возвращает новый pd.DataFrame с двумя колонками: app_id и sequences.
    каждое значение в колонке sequences - это список списков.
    каждый список - значение одного конкретного признака во всех клиентских транзакциях.
    Всего признаков len(features), поэтому будет len(features) списков.
    Данная функция крайне полезна для подготовки датасета для работы с нейронными сетями.
    :param transactions_frame: фрейм с транзакциями клиентов
    :param num_last_transactions: количество транзакций клиента, которые будут рассмотрены
    :return: pd.DataFrame из двух колонок (app_id, sequences)
    """
    return transactions_frame \
        .sort_values(['app_id', 'transaction_number']) \
        .groupby(['app_id'])[features] \
        .apply(lambda x: truncate(x, num_last_transactions=num_last_transactions)) \
        .reset_index().rename(columns={0: 'sequences'})

### Create padded buckets

In [6]:
def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path=None, has_target=True):
    """
    Функция реализует sequence_bucketing технику для обучения нейронных сетей.
    Принимает на вход frame_of_sequences (результат работы функции transform_transactions_to_sequences),
    словарь bucket_info, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding, далее группирует транзакции по бакетам (на основе длины), делает padding транзакций и сохраняет результат
    в pickle файл, если нужно
    :param frame_of_sequences: pd.DataFrame c транзакциями (результат применения transform_transactions_to_sequences)
    :param bucket_info: словарь, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding
    :param save_to_file_path: опциональный путь до файла, куда нужно сохранить результат
    :param has_target: флаг, есть ли в frame_of_sequences целевая переменная или нет. Если есть, то
    будет записано в результат
    :return: возвращает словарь с следюущими ключами (padded_sequences, targets, app_id, products)
    """
    frame_of_sequences['bucket_idx'] = frame_of_sequences.sequence_length.map(bucket_info)
    padded_seq = []
    targets = []
    app_ids = []
    products = []

    for size, bucket in frame_of_sequences.groupby('bucket_idx'):
        padded_sequences = bucket.sequences.apply(lambda x: pad_sequence(x, size)).values
        padded_sequences = np.array([np.array(x) for x in padded_sequences])
        padded_seq.append(padded_sequences)

        if has_target:
            targets.append(bucket.flag.values)

        app_ids.append(bucket.app_id.values)
        products.append(bucket['product'].values)

    frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)

    dict_result = {
        'padded_sequences': np.array(padded_seq, dtype=object),
        'targets': np.array(targets, dtype=object) if targets else [],
        'app_id': np.array(app_ids, dtype=object),
        'products': np.array(products, dtype=object),
    }

    if save_to_file_path:
        with open(save_to_file_path, 'wb') as f:
            pickle.dump(dict_result, f)
    return dict_result

In [7]:
import pickle

with open('../input/alfabattle2-sandbox/constants_for_rnn/constants_for_rnn/buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('../input/alfabattle2-sandbox/constants_for_rnn/constants_for_rnn/dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

In [8]:
def create_buckets_from_transactions(path_to_dataset, save_to_path, frame_with_ids = None, 
                                     num_parts_to_preprocess_at_once: int = 1, 
                                     num_parts_total=50, has_target=False):
    """
    Функция `create_buckets_from_transactions` ниже реализует следующий набор действий:
    * Читает `num_parts_to_preprocess_at_once` частей датасета в память
    * Преобразует вещественные и численные признаки к категориальным (используя `np.digitize` и подготовленные бины)
    * Формирует фрейм с транзакциями в виде последовательностей с помощью `transform_transactions_to_sequences`.
    * Если указан `frame_with_ids`, то использует `app_id` из `frame_with_ids` - актуально, чтобы выделить валидационную выборку.
    * Реализует технику `sequence_bucketing` и сохраняет словарь обработанных последовательностей в `.pkl` файл
    """
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, 
                                                             verbose=True)
        for dense_col in ['amnt', 'days_before', 'hour_diff']:
            transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
            
        seq = transform_transactions_to_sequences(transactions_frame)
        seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on='app_id')

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = '00' + block_as_str
        else:
            block_as_str = '0' + block_as_str
            how
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f'processed_chunk_{block_as_str}.pkl'))
        block += 1

* Разобьем имеющиеся данные на `train` и `val` части. Воспользуемся самым простым способом - для валидации используем 10% случайных данных

In [9]:
path_to_dataset = '../input/train-val-buckets/val_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

['../input/train-val-buckets/val_buckets/processed_chunk_000.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_001.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_002.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_003.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_004.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_005.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_006.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_007.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_008.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_009.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_010.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_011.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_012.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_013.pkl',
 '../input/train-val-buckets/val_buckets/processed_chunk_014.p

In [10]:
path_to_dataset = '../input/train-val-buckets/train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_train

['../input/train-val-buckets/train_buckets/processed_chunk_000.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_001.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_002.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_003.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_004.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_005.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_006.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_007.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_008.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_009.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_010.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_011.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_012.pkl',
 '../input/train-val-buckets/train_buckets/processed_chunk_013.pkl',
 '../input/train-val-buckets/train

### 2. Modeling

    * `data_generators.batches_generator` - функция-генератор, итеративно возвращает батчи, поддерживает батчи для `tensorflow.keras` и `torch.nn.module` моделей. В зависимости от флага `is_train` может быть использована для генерации батчей на train/val/test стадию.
    * функция `pytorch_training.train_epoch` - обучает модель одну эпоху.
    * функция `pytorch_training.eval_model` - проверяет качество модели на отложенной выборке и возвращает roc_auc_score.
    * функция `pytorch_training.inference` - делает предикты на новых данных и готовит фрейм для проверяющей системы.
    * класс `training_aux.EarlyStopping` - реализует early_stopping, сохраняя лучшую модель. Пример использования приведен ниже.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [12]:
transaction_features = ['currency', 'operation_kind', 'card_type', 'operation_type',
                        'operation_type_group', 'ecommerce_flag', 'payment_system',
                        'income_flag', 'mcc', 'country', 'city', 'mcc_category',
                        'day_of_week', 'hour', 'weekofyear', 'amnt', 'days_before', 'hour_diff']

def batches_generator(list_of_paths, batch_size=32, shuffle=False, is_infinite=False,
                      verbose=False, device=None, output_format='torch', is_train=True):
    """
    функция для создания батчей на вход для нейронной сети для моделей на keras и pytorch.
    так же может использоваться как функция на стадии инференса
    :param list_of_paths: путь до директории с предобработанными последовательностями
    :param batch_size: размер батча
    :param shuffle: флаг, если True, то перемешивает list_of_paths и так же
    перемешивает последовательности внутри файла
    :param is_infinite: флаг, если True,  то создает бесконечный генератор батчей
    :param verbose: флаг, если True, то печатает текущий обрабатываемый файл
    :param device: device на который положить данные, если работа на торче
    :param output_format: допустимые варианты ['tf', 'torch']. Если 'torch', то возвращает словарь,
    где ключи - батчи из признаков, таргетов и app_id. Если 'tf', то возвращает картеж: лист input-ов
    для модели, и список таргетов.
    :param is_train: флаг, Если True, то для кераса вернет (X, y), где X - input-ы в модель, а y - таргеты, 
    если False, то в y будут app_id; для torch вернет словарь с ключами на device.
    :return: бачт из последовательностей и таргетов (или app_id)
    """
    while True:
        if shuffle:
            np.random.shuffle(list_of_paths)

        for path in list_of_paths:
            if verbose:
                print(f'reading {path}')

            with open(path, 'rb') as f:
                '''
                26.pkl is truncated 
                '''
                if path == '../input/train-val-buckets/train_buckets/processed_chunk_026.pkl':
                    continue
                data = pickle.load(f)
            padded_sequences, targets, products = data['padded_sequences'], data['targets'], data[
                'products']
            app_ids = data['app_id']
            indices = np.arange(len(products))

            if shuffle:
                np.random.shuffle(indices)
                padded_sequences = padded_sequences[indices]
                targets = targets[indices]
                products = products[indices]
                app_ids = app_ids[indices]

            for idx in range(len(products)):
                bucket, product = padded_sequences[idx], products[idx]
                app_id = app_ids[idx]
                
                if is_train:
                    target = targets[idx]
                
                for jdx in range(0, len(bucket), batch_size):
                    batch_sequences = bucket[jdx: jdx + batch_size]
                    if is_train:
                        batch_targets = target[jdx: jdx + batch_size]
                    
                    batch_products = product[jdx: jdx + batch_size]
                    batch_app_ids = app_id[jdx: jdx + batch_size]
                    
                    if output_format == 'tf':
                        batch_sequences = [batch_sequences[:, i] for i in
                                           range(len(transaction_features))]
                        
                        # append product as input to tf model
                        batch_sequences.append(batch_products)
                        if is_train:
                            yield batch_sequences, batch_targets
                        else:
                             yield batch_sequences, batch_app_ids
                    else:
                        batch_sequences = [torch.LongTensor(batch_sequences[:, i]).to(device)
                                           for i in range(len(transaction_features))]
                        if is_train:
                            yield dict(transactions_features=batch_sequences,
                                       product=torch.LongTensor(batch_products).to(device),
                                       label=torch.LongTensor(batch_targets).to(device),
                                       app_id=batch_app_ids)
                        else:
                            yield dict(transactions_features=batch_sequences,
                                       product=torch.LongTensor(batch_products).to(device),
                                       app_id=batch_app_ids)
        if not is_infinite:
            break

In [13]:
def train_epoch(model, optimizer, dataset_train, batch_size=64, shuffle=True,
                print_loss_every_n_batches=500, device=None):
    """
    делает одну эпоху обучения модели, логирует
    :param model: nn.Module модель
    :param optimizer: nn.optim оптимизатор
    :param dataset_train: путь до директории с последовательностями
    :param batch_size: размерм батча
    :param shuffle: флаг, если True, то перемешивает данные
    :param print_loss_every_n_batches: число батчей после которых логируется лосс на этих батчах
    :param device: device, на который будут положены данные внутри батча
    :return: None
    """
    train_generator = batches_generator(dataset_train, batch_size=batch_size, shuffle=shuffle,
                                        device=device, is_train=True, output_format='torch')
    loss_function = nn.BCEWithLogitsLoss()
    
    num_batches = 1
    running_loss = 0.0
    max_grad_norm = 1000
    
    model.train()

    for batch in tqdm(train_generator, desc='Training'):

        output = torch.flatten(model(batch['transactions_features'], batch['product']))

        batch_loss = loss_function(output, batch['label'].float())
        
        batch_loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()

        running_loss += batch_loss

        if num_batches % print_loss_every_n_batches == 0:
            print(f'Training loss after {num_batches} batches: {running_loss / num_batches}', end='\r')
        
        num_batches += 1
    
    print(f'Training loss after epoch: {running_loss / num_batches}', end='\r')

In [14]:
def batches_complete_generator():
    '''
    reinitialize new generator which union incomplete batches and after split this into full batches
    '''
    tr_generator = batches_generator([dataset_train[0]], batch_size=128, shuffle=False,
                        device=device, is_train=True, output_format='torch')
    
    incomplete_batches = {}
    
    for idx, batch in enumerate(tr_generator):
        if batch['transactions_features'][0].shape[0] != batch_size:
            incomplete_batches[batch['transactions_features'][0].shape[1]] = incomplete_batches.get(batch['transactions_features'][0].shape[1], []).append(batch)
        else:
            yield batch
            
    split_batches = {
        'padded_sequences': [],
        'targets': [],
        'app_id': [],
        'products': [],
    }
    
    for key_bathes, val_batches in incomplete_batches.items():
        for key in split_batches:
            concatenate_batches = torch.cat(*(batch['padded_sequences'] for batch in val_batches), dim=0)
            split_batches[key] = torch.split(concatenate_batches, split_size_or_sections=batch_size)
            
        for idx in range(len(split_batches['padded_sequences'])):
            yield {'padded_sequences': split_batches['padded_sequences'][idx],
                    'targets': split_batches['targets'][idx],
                    'app_id': split_batches['app_id'][idx],
                    'products': split_batches['products'][idx]} 

In [15]:

def eval_model(model, dataset_val, batch_size=32, device=None) -> float:
    """
    функция для оценки качества модели на отложенной выборке, возвращает roc-auc на валидационной
    выборке
    :param model: nn.Module модель
    :param dataset_val: путь до директории с последовательностями
    :param batch_size: размер батча
    :param device: device, на который будут положены данные внутри батча
    :return: val roc-auc score
    """
    preds = []
    targets = []
    val_generator = batches_generator(dataset_val, batch_size=batch_size, shuffle=False,
                                      device=device, is_train=True, output_format='torch')
    model.eval()

    for batch in tqdm(val_generator, desc='Evaluating model'):
        targets.extend(batch['label'].detach().cpu().numpy().flatten())
        output = model(batch['transactions_features'], batch['product'])
        preds.extend(output.detach().cpu().numpy().flatten())

    return roc_auc_score(targets, preds)


In [16]:
def inference(model, dataset_test, batch_size=32, device=None) -> pd.DataFrame:
    """
    функция, которая делает предикты на новых данных, возвращает pd.DataFrame из двух колонок:
    (app_id, score)
    :param model: nn.Module модель
    :param dataset_test: путь до директории с последовательностями
    :param batch_size: размер батча
    :param device: device, на который будут положены данные внутри батча
    :return: pd.DataFrame из двух колонок: (app_id, score)
    """
    model.eval()
    preds = []
    app_ids = []
    test_generator = batches_generator(dataset_test, batch_size=batch_size, shuffle=False,
                                       verbose=False, device=device, is_train=False,
                                       output_format='torch')

    for batch in tqdm(test_generator, desc='Test time predictions'):
        app_ids.extend(batch['app_id'])
        output = model(batch['transactions_features'], batch['product'])
        preds.extend(output.detach().cpu().numpy().flatten())
        
    return pd.DataFrame({
        'app_id': app_ids,
        'score': preds
    })

In [17]:
class EarlyStopping:
    """Early stops the training if validation metric doesn't improve after a given patience."""

    def __init__(self, patience=7, mode='min', verbose=False, delta=0, save_path='checkpoint.hdf5', metric_name=None, save_format='torch'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            mode (str): One of ['min', 'max'], whether to maximize or minimaze the metric.
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            save_path (str): Path to saved model
        """
        if mode not in ['min', 'max']:
            raise ValueError(f'Unrecognized mode: {mode}!')

        self.patience = patience
        self.mode = mode
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_prev_score = np.Inf if mode == 'min' else -np.Inf
        self.delta = delta
        self.save_path = save_path
        self.metric_name = 'metric' if not metric_name else metric_name
        if save_format not in ['torch', 'tf']:
            raise ValueError('Expected to save in one of the following formats: ["torch", "tf"]')
        self.save_format = save_format
        
    def __call__(self, metric_value, model):

        score = -metric_value if self.mode == 'min' else metric_value

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(metric_value, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                f'No imporvement in Validation {self.metric_name}. Current: {score:.6f}. Current best: {self.best_score:.6f}')
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(metric_value, model)
            self.counter = 0

    def save_checkpoint(self, metric_value, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f'Validation {self.metric_name} improved ({self.best_prev_score:.6f} --> {metric_value:.6f}).  Saving model ...')
        if self.save_format == 'tf':
            model.save_weights(self.save_path)
        else:
            torch.save(model.state_dict(), self.save_path)
            
        self.best_prev_score = metric_value

* Все признаки в нашей модели будут категориальными. Для их представления в модели используем категориальные эмбеддинги. Для этого нужно каждому категориальному признаку задать размерность латентного пространства. Используем [формулу](https://forums.fast.ai/t/size-of-embedding-for-categorical-variables/42608) из библиотеки `fast.ai`. Все отображения хранятся в файле `embedding_projections.pkl`

In [18]:
with open('../input/alfabattle2-sandbox/constants_for_rnn/constants_for_rnn/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)

* Реализуем модель. Все входные признаки представим в виде эмбеддингов, сконкатенируем, чтобы получить векторное представление транзакции. Подадим последовательности в `GRU` рекуррентную сеть. Используем последнее скрытое состояние в качестве выхода сети. Представим признак `product` в виде отдельного эмбеддинга. Сконкатенируем его с выходом сети. На основе такого входа построим небольшой `MLP`, выступающий классификатором для целевой задачи. Используем градиентный спуск, чтобы решить оптимизационную задачу. 

In [19]:
class TransactionsRnn(nn.Module):
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', rnn_units=128, top_classifier_units=32):
        super(TransactionsRnn, self).__init__()
        self._transaction_cat_embeddings = nn.ModuleList([self._create_embedding_projection(*embedding_projections[feature]) 
                                                          for feature in transactions_cat_features])
                
        self._product_embedding = self._create_embedding_projection(*embedding_projections[product_col_name], padding_idx=None)
        
        self._gru = nn.GRU(input_size=sum([embedding_projections[x][1] for x in transactions_cat_features]),
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=False)
        
        self._hidden_size = rnn_units
        
        self._top_classifier = nn.Linear(in_features=rnn_units+embedding_projections[product_col_name][1], 
                                         out_features=top_classifier_units)
        self._intermediate_activation = nn.ReLU()
        
        self._head = nn.Linear(in_features=top_classifier_units, out_features=1)
    
    def forward(self, transactions_cat_features, product_feature):
        batch_size = product_feature.shape[0]
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        
        concated_embeddings = torch.cat(embeddings, dim=-1)
        
        _, last_hidden = self._gru(concated_embeddings)
        
        last_hidden = torch.reshape(last_hidden.permute(1, 2, 0), shape=(batch_size, self._hidden_size))
        
        product_embed = self._product_embedding(product_feature)

        intermediate_concat = torch.cat([last_hidden, product_embed], dim=-1)
  
        classification_hidden = self._top_classifier(intermediate_concat)

        activation = self._intermediate_activation(classification_hidden)
        
        logit = self._head(activation)
        
        return logit
    
    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(num_embeddings=cardinality+add_missing, embedding_dim=embed_size, padding_idx=padding_idx)


In [20]:
class BiGRU2FCL(TransactionsRnn):
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', rnn_units=128, top_classifier_units=32):
        super(BiGRU2FCL, self).__init__(transactions_cat_features, embedding_projections, product_col_name, rnn_units, top_classifier_units)

        self._gru = nn.GRU(input_size=sum([embedding_projections[x][1] for x in transactions_cat_features]),
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=True)
        
        self._hidden_size = rnn_units*2
        
        self._top_classifier = nn.Linear(in_features=rnn_units*2+embedding_projections[product_col_name][1], 
                                         out_features=top_classifier_units*2)
        
        self._head = nn.Linear(in_features=top_classifier_units*2, out_features=1)
    
    

In [21]:
class BiGRU2FCL_Norm(BiGRU2FCL):
    '''
    add BatchNorm layer before FCL
    '''
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', rnn_units=128, top_classifier_units=32):
        super(BiGRU2FCL_Norm, self).__init__(transactions_cat_features, embedding_projections, product_col_name, rnn_units, top_classifier_units)
        self._batch_norm = nn.BatchNorm1d(num_features=self._hidden_size+embedding_projections[product_col_name][1])
        
    def forward(self, transactions_cat_features, product_feature):
        batch_size = product_feature.shape[0]
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        
        concated_embeddings = torch.cat(embeddings, dim=-1)
        
        _, last_hidden = self._gru(concated_embeddings)
        
        last_hidden = torch.reshape(last_hidden.permute(1, 2, 0), shape=(batch_size, self._hidden_size))
        
        product_embed = self._product_embedding(product_feature)
        
        intermediate_concat = torch.cat([last_hidden, product_embed], dim=-1)
        
        norm_intermediate_concat = self._batch_norm(intermediate_concat)
            
        classification_hidden = self._top_classifier(norm_intermediate_concat)

        activation = self._intermediate_activation(classification_hidden)
        
        logit = self._head(activation)
        
        return logit

### 3. Training

#### Work Pipeline
* RNN: variational DropOut
* redesign batch generation
* RNN regulirization: forget_gate ~ 1, gradient cliping
* build CNN + LSTM + CRF model in tensorflow
* analyse advanced baseline

In [22]:
! mkdir ./rnn_baseline

! mkdir ./rnn_baseline/checkpoints

! rm -r ./rnn_baseline/checkpoints/pytorch_baseline
! mkdir ./rnn_baseline/checkpoints/pytorch_baseline

rm: cannot remove './rnn_baseline/checkpoints/pytorch_baseline': No such file or directory


* Для того, чтобы детектировать переобучение используем EarlyStopping.

In [23]:
path_to_checkpoints = './rnn_baseline/checkpoints/pytorch_baseline/'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='torch')

In [24]:
num_epochs = 10
train_batch_size = 128
val_batch_szie = 128

In [25]:
#model = TransactionsRnn(transaction_features, embedding_projections).to(device)

In [26]:
model = BiGRU2FCL(transaction_features, embedding_projections).to(device)

In [27]:
model

BiGRU2FCL(
  (_transaction_cat_embeddings): ModuleList(
    (0): Embedding(12, 6, padding_idx=0)
    (1): Embedding(8, 5, padding_idx=0)
    (2): Embedding(176, 29, padding_idx=0)
    (3): Embedding(23, 9, padding_idx=0)
    (4): Embedding(5, 3, padding_idx=0)
    (5): Embedding(4, 3, padding_idx=0)
    (6): Embedding(8, 5, padding_idx=0)
    (7): Embedding(4, 3, padding_idx=0)
    (8): Embedding(109, 22, padding_idx=0)
    (9): Embedding(25, 9, padding_idx=0)
    (10): Embedding(164, 28, padding_idx=0)
    (11): Embedding(29, 10, padding_idx=0)
    (12): Embedding(8, 5, padding_idx=0)
    (13): Embedding(25, 9, padding_idx=0)
    (14): Embedding(54, 15, padding_idx=0)
    (15): Embedding(11, 6, padding_idx=0)
    (16): Embedding(24, 9, padding_idx=0)
    (17): Embedding(11, 6, padding_idx=0)
  )
  (_product_embedding): Embedding(6, 4)
  (_gru): GRU(182, 128, batch_first=True, bidirectional=True)
  (_top_classifier): Linear(in_features=260, out_features=64, bias=True)
  (_intermediate_

In [28]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

* Запустим цикл обучения, каждую эпоху будем логировать лосс, а так же roc-auc на валидации и на обучении. Будем сохрнаять веса после каждой эпохи, а так же лучшие с помощью early_stopping.

In [29]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    train_epoch(model, optimizer, dataset_train, batch_size=train_batch_size, 
                shuffle=True, print_loss_every_n_batches=500, device=device)
    
    val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie, device=device)
    es(val_roc_auc, model)
    scheduler.step(val_roc_auc)
    
    if es.early_stop:
        print('Early stopping reached. Stop training...')
        break
    torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))
    
    train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
    print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

Starting epoch 1


Training: 505it [00:34, 29.51it/s]

Training: 1003it [01:07, 15.36it/s]

Training: 1503it [01:35, 39.97it/s]

Training: 2001it [02:09, 11.18it/s]

...

Training: 4502it [04:03, 22.33it/s]

Training: 5003it [04:30, 19.38it/s]

Training: 5062it [04:32, 18.57it/s]


Evaluating model: 5033it [01:12, 69.53it/s]

No imporvement in Validation ROC-AUC. Current: 0.729631. Current best: 0.766579
EarlyStopping counter: 3 out of 3
Early stopping reached. Stop training...
